### Preprocessing financial statements using Gemini

In this notebook, I preprocess the downloaded reports as far as required in order to feed them into the LLM to obtain a buy/sell recommendation.
Reuters uses abbreviations for all positions in financial statements. Using Gemini to infer the full position name considerably helps the process of preparing the dataset.

In [1]:
import pandas as pd
import numpy as np 
from google import genai
from google.genai import types
import json
import re
from tqdm import tqdm
import time
from google.genai.errors import ServerError  

In [ ]:
# Import Gemini API key
with open("../proton_google_api_key.txt", "r") as f:
    key = f.read().strip()    

# Initialize the Gemini client with the API key
client = genai.Client(api_key = key)

In [15]:
# Read in company names to look up buy/sell recommendations for
sp400_companies = pd.read_csv("../data/sp400_companies.csv", dtype={"CIK": str})
sp500_companies = pd.read_csv("../data/sp500_companies.csv", dtype={"CIK": str})
sp600_companies = pd.read_csv("../data/sp600_companies.csv", dtype={"CIK": str})

# Combine all CIKs into a single list
sp400_ciks = sp400_companies["CIK"].tolist()
sp500_ciks = sp500_companies["CIK"].tolist()
sp600_ciks = sp600_companies["CIK"].tolist()
ciks = sp400_ciks + sp500_ciks + sp600_ciks

# Read in RICs to match with CIKs
rics = pd.read_csv("../data/rics.csv", dtype={"Instrument": str})
# Ranem Instrument to CIK for easier merging
rics.rename(columns={"Instrument": "CIK"}, inplace=True)

- Import and further process balance sheet df

In [ ]:
first_balance_sheets = pd.read_csv("../data/balance_sheets.csv", dtype={"CIK": str})
missing_balance_sheets = pd.read_csv("../data/missing_balance_sheets.csv", dtype={"CIK": str})
balance_sheets = pd.concat([first_balance_sheets, missing_balance_sheets], ignore_index=True)
# Delete entries where STD Balance Sheet All and FCC Item Name are missing
balance_sheets = balance_sheets.dropna(subset=["STD Balance Sheet All", "FCC Item Name"])
balance_sheets.head()

,Date,STD Balance Sheet All,FCC Item Name,RIC,Statement
4,2016-06-30,332000000.0,TR.F.CashSTInvst,AA.N,balance_sheet
5,2016-06-30,332000000.0,TR.F.CashCashEquiv,AA.N,balance_sheet
6,2016-06-30,74000000.0,TR.F.DerivFinInstrHedgeST,AA.N,balance_sheet
7,2016-06-30,607000000.0,TR.F.LoansRcvblNetST,AA.N,balance_sheet
8,2016-06-30,426000000.0,TR.F.TradeAcctTradeNotesRcvblNet,AA.N,balance_sheet


In [5]:
# Adjust Item Name to give a clearer indication of the position
unique_balance_sheet_positions = balance_sheets["FCC Item Name"].unique()

In this cell, I employ Gemini to return actual official balance sheet position labels for the Thomson Reuters abbreviations contained in financial statements downloaded via the LSEG Data Library for Python.

In [ ]:
balance_sheet_labels = client.models.generate_content(
    model="gemini-2.5-flash",
    config=types.GenerateContentConfig(
        temperature=0.1,
        system_instruction="You are a financial analyst, that provides concise and accurate answers.",
        thinking_config=types.ThinkingConfig(thinking_budget=0),# Disables thinking
    ),
    contents=[f"""The following is a list of abbreviations used by Thomsom Reuters in their reporting database. For every given abbreviation,
              please provide the actual financial position this refers to in a balance sheet. Example: TR.F.TotAssets: Total Assets.
              Here are the abbreviations: {unique_balance_sheet_positions.tolist()} Provide your answer in a dictionary style format.
              """],  
)

In [ ]:
# Regex to extract JSON from the response
# First I extract the JSON string from the response text
json_str = re.search(r'{.*}', balance_sheet_labels.text, re.DOTALL).group()
# Then I parse the JSON string into a Python dictionary
data = json.loads(json_str)
# Save the dictionary to a file
with open("../data/balance_sheet_labels.json", "w") as f:
    json.dump(data, f, indent=4)  # indent makes JSON-format more readable

In [16]:
# Finally, this dictionary can be used to map the abbreviations to the actual financial positions in the balance sheets.
balance_sheets["position_label"] = balance_sheets["FCC Item Name"].map(data)

In [17]:
# Merge on RIC to include CIK and RIC in the balance sheets
balance_sheets = balance_sheets.merge(rics, left_on="RIC", right_on="RIC", how="left")
balance_sheets.head(10)

,Date,STD Balance Sheet All,FCC Item Name,RIC,Statement,position_label,CIK
0,2016-06-30,3.320000e+08,TR.F.CashSTInvst,AA.N,balance_sheet,Cash and Short-Term Investments,0001675149
1,2016-06-30,3.320000e+08,TR.F.CashCashEquiv,AA.N,balance_sheet,Cash and Cash Equivalents,0001675149
2,2016-06-30,7.400000e+07,TR.F.DerivFinInstrHedgeST,AA.N,balance_sheet,"Derivative Financial Instruments, Hedging, Sho...",0001675149
3,2016-06-30,6.070000e+08,TR.F.LoansRcvblNetST,AA.N,balance_sheet,"Loans and Receivables, Net, Short-Term",0001675149
4,2016-06-30,4.260000e+08,TR.F.TradeAcctTradeNotesRcvblNet,AA.N,balance_sheet,"Trade Accounts and Trade Notes Receivable, Net",0001675149
5,2016-06-30,1.810000e+08,TR.F.RcvblOthTot,AA.N,balance_sheet,"Receivables, Other, Total",0001675149
6,2016-06-30,1.166000e+09,TR.F.InvntTot,AA.N,balance_sheet,"Inventory, Total",0001675149
7,2016-06-30,8.350000e+08,TR.F.InvntRawMaterial,AA.N,balance_sheet,"Inventory, Raw Material",0001675149
8,2016-06-30,1.760000e+08,TR.F.InvntWorkInProg,AA.N,balance_sheet,"Inventory, Work In Progress",0001675149
9,2016-06-30,1.550000e+08,TR.F.InvntFinGoods,AA.N,balance_sheet,"Inventory, Finished Goods",0001675149


In [ ]:
# Save to CSV
balance_sheets.to_csv("../data/balance_sheets_with_labels.csv", index=False)

---
- Import and further process Cash Flow statement df

In [ ]:
first_cash_flow_statements = pd.read_csv("../data/cash_flow_statements.csv", dtype={"CIK": str})
missing_cash_flow_statements = pd.read_csv("../data/missing_cash_flow_statements.csv", dtype={"CIK": str})
cash_flow_statements = pd.concat([first_cash_flow_statements, missing_cash_flow_statements], ignore_index=True)
cash_flow_statements.head()

,Date,STD Cash Flow All,FCC Item Name,RIC,statement,Statement
0,2011-09-30,NaN,NaN,AA.N,cashflow,NaN
1,2012-09-30,NaN,NaN,AA.N,cashflow,NaN
2,2013-09-30,NaN,NaN,AA.N,cashflow,NaN
3,2014-09-30,NaN,NaN,AA.N,cashflow,NaN
4,2016-06-30,-19000000.0,TR.F.ProfLossStartingLineCF,AA.N,cashflow,NaN


In [5]:
# Fix wrong column names
cash_flow_statements = cash_flow_statements.drop(columns = "Statement")
cash_flow_statements.rename(columns={"statement": "Statement"}, inplace=True)
# Delete entries where both STD Cash Flow All and FCC Item Name is NaN
cash_flow_statements = cash_flow_statements.dropna(subset=["STD Cash Flow All", "FCC Item Name"], how="all")
cash_flow_statements.head()

,Date,STD Cash Flow All,FCC Item Name,RIC,Statement
4,2016-06-30,-19000000.0,TR.F.ProfLossStartingLineCF,AA.N,cashflow
5,2016-06-30,245000000.0,TR.F.NonCashItemsReconcAdjCF,AA.N,cashflow
6,2016-06-30,16000000.0,TR.F.EqIncLossInNetEarnCF,AA.N,cashflow
7,2016-06-30,103000000.0,TR.F.OthNonCashItemsReconcAdjCF,AA.N,cashflow
8,2016-06-30,178000000.0,TR.F.DeprDeplAmortInclImpairCF,AA.N,cashflow


In [6]:
# Get unique cash flow positions
unique_cash_flow_positions = cash_flow_statements["FCC Item Name"].unique()

- As before, Gemini is employed to infer the correct cash flow labels from the FCC Item Name Codes

In [9]:
cash_flow_labels = client.models.generate_content(
    model="gemini-2.5-flash",
    config=types.GenerateContentConfig(
        temperature=0.1,
        system_instruction="You are a financial analyst, that provides concise and accurate answers.",
        thinking_config=types.ThinkingConfig(thinking_budget=0),# Disables thinking
    ),
    contents=[f"""The following is a list of abbreviations used by Thomsom Reuters in their reporting database. For every given abbreviation,
              please provide the actual financial position this refers to in a cash flow statement. Example: TR.F.TotAssets: Total Assets.
              Here are the abbreviations: {unique_cash_flow_positions.tolist()} Provide your answer in a dictionary style format.
              """],  
)

In [10]:
# Regex to extract JSON from the response
# First I extract the JSON string from the response text
json_str = re.search(r'{.*}', cash_flow_labels.text, re.DOTALL).group()
# Then I parse the JSON string into a Python dictionary
data = json.loads(json_str)
# Save the dictionary to a file
with open("data/cash_flow_labels.json", "w") as f:
    json.dump(data, f, indent=4)  

In [11]:
# Map dictionary to cash flow statements
cash_flow_statements["position_label"] = cash_flow_statements["FCC Item Name"].map(data)

In [12]:
# Merge on RIC to include CIK and RIC in the cash flow statements
cash_flow_statements = cash_flow_statements.merge(rics, left_on="RIC", right_on="RIC", how="left")
cash_flow_statements.head(10)

,Date,STD Cash Flow All,FCC Item Name,RIC,Statement,position_label,CIK
0,2016-06-30,-19000000.0,TR.F.ProfLossStartingLineCF,AA.N,cashflow,Profit/Loss Starting Line (Cash Flow),0001675149
1,2016-06-30,245000000.0,TR.F.NonCashItemsReconcAdjCF,AA.N,cashflow,Non-Cash Items Reconciliation Adjustments (Cas...,0001675149
2,2016-06-30,16000000.0,TR.F.EqIncLossInNetEarnCF,AA.N,cashflow,Equity Income/Loss in Net Earnings (Cash Flow),0001675149
3,2016-06-30,103000000.0,TR.F.OthNonCashItemsReconcAdjCF,AA.N,cashflow,Other Non-Cash Items Reconciliation Adjustment...,0001675149
4,2016-06-30,178000000.0,TR.F.DeprDeplAmortInclImpairCF,AA.N,cashflow,"Depreciation, Depletion, Amortization, Includi...",0001675149
5,2016-06-30,178000000.0,TR.F.DeprDeplPPECF,AA.N,cashflow,"Depreciation, Depletion of Property, Plant, an...",0001675149
6,2016-06-30,-28000000.0,TR.F.DefIncTaxIncTaxCreditsCF,AA.N,cashflow,Deferred Income Tax and Income Tax Credits (Ca...,0001675149
7,2016-06-30,-34000000.0,TR.F.AssetsSaleGLCF,AA.N,cashflow,Assets Sale Gain/Loss (Cash Flow),0001675149
8,2016-06-30,10000000.0,TR.F.ShrBasedPaymtCF,AA.N,cashflow,Share-Based Payment (Cash Flow),0001675149
9,2016-06-30,226000000.0,TR.F.CashFlowOpBefChgInWkgCap,AA.N,cashflow,Cash Flow from Operations Before Changes in Wo...,0001675149


In [ ]:
# Save to CSV
cash_flow_statements.to_csv("../data/cash_flow_statements_with_labels.csv", index=False)

---
- Import and further process income statements

In [ ]:
first_income_statements = pd.read_csv("../data/income_statements.csv", dtype={"CIK": str})
missing_income_statements = pd.read_csv("../data/missing_income_statements.csv", dtype={"CIK": str})
income_statements = pd.concat([first_income_statements, missing_income_statements], ignore_index=True)
income_statements.head()

,Date,STD Income Statement All,FCC Item Name,RIC,statement,Statement
0,2011-09-30,NaN,NaN,AA.N,income_statement,NaN
1,2012-09-30,NaN,NaN,AA.N,income_statement,NaN
2,2013-09-30,NaN,NaN,AA.N,income_statement,NaN
3,2014-09-30,NaN,NaN,AA.N,income_statement,NaN
4,2016-06-30,2.323000e+09,TR.F.RevGoodsSrvc,AA.N,income_statement,NaN


In [23]:
# Drop observations, where both STD Income Statement All and FCC Item Name is NaN
income_statements = income_statements.dropna(subset=["STD Income Statement All", "FCC Item Name"], how="all")
# Get rid of Statement column, which is result of wrong spelling
income_statements = income_statements.drop(columns = "Statement")

In [17]:
# Unique positions in income statements
unique_income_positions = income_statements["FCC Item Name"].unique()

- One last time, Gemini is employed to map the FCC Item Name Codes to more official labels used in Cash Flow statements

In [18]:
income_statement_labels = client.models.generate_content(
    model="gemini-2.5-flash",
    config=types.GenerateContentConfig(
        temperature=0.1,
        system_instruction="You are a financial analyst, that provides concise and accurate answers.",
        thinking_config=types.ThinkingConfig(thinking_budget=0),# Disables thinking
    ),
    contents=[f"""The following is a list of abbreviations used by Thomsom Reuters in their reporting database. For every given abbreviation,
              please provide the actual financial position this refers to in an income statement. Example: TR.F.TotAssets: Total Assets.
              Here are the abbreviations: {unique_income_positions.tolist()} Provide your answer in a dictionary style format.
              """],  
)

In [ ]:
# Regex to extract JSON from the response
# First I extract the JSON string from the response text
json_str = re.search(r'{.*}', income_statement_labels.text, re.DOTALL).group()
# Then I parse the JSON string into a Python dictionary
data = json.loads(json_str)
# Save the dictionary to a file
with open("../data/income_statement_labels.json", "w") as f:   
    json.dump(data, f, indent=4)

In [24]:
# Map dictionary to cash flow statements
income_statements["position_label"] = income_statements["FCC Item Name"].map(data)
income_statements.head()

,Date,STD Income Statement All,FCC Item Name,RIC,statement,position_label
4,2016-06-30,2.323000e+09,TR.F.RevGoodsSrvc,AA.N,income_statement,Revenue from Goods and Services
5,2016-06-30,2.323000e+09,TR.F.SalesOfGoodsSrvcNetUnclassif,AA.N,income_statement,"Sales of Goods and Services, Net, Unclassified"
6,2016-06-30,2.323000e+09,TR.F.TotRevenue,AA.N,income_statement,Total Revenue
7,2016-06-30,2.119000e+09,TR.F.CostOfOpRev,AA.N,income_statement,Cost of Operating Revenue
8,2016-06-30,2.119000e+09,TR.F.COGSTot,AA.N,income_statement,"Cost of Goods Sold, Total"


In [25]:
# Merge on RIC to include CIK and RIC in the cash flow statements
income_statements = income_statements.merge(rics, left_on="RIC", right_on="RIC", how="left")
income_statements.head(10)

,Date,STD Income Statement All,FCC Item Name,RIC,statement,position_label,CIK
0,2016-06-30,2.323000e+09,TR.F.RevGoodsSrvc,AA.N,income_statement,Revenue from Goods and Services,0001675149
1,2016-06-30,2.323000e+09,TR.F.SalesOfGoodsSrvcNetUnclassif,AA.N,income_statement,"Sales of Goods and Services, Net, Unclassified",0001675149
2,2016-06-30,2.323000e+09,TR.F.TotRevenue,AA.N,income_statement,Total Revenue,0001675149
3,2016-06-30,2.119000e+09,TR.F.CostOfOpRev,AA.N,income_statement,Cost of Operating Revenue,0001675149
4,2016-06-30,2.119000e+09,TR.F.COGSTot,AA.N,income_statement,"Cost of Goods Sold, Total",0001675149
5,2016-06-30,1.941000e+09,TR.F.COGSUnclassif,AA.N,income_statement,"Cost of Goods Sold, Unclassified",0001675149
6,2016-06-30,1.780000e+08,TR.F.DeprInCOGS,AA.N,income_statement,Depreciation in Cost of Goods Sold,0001675149
7,2016-06-30,2.040000e+08,TR.F.GrossProfIndPropTot,AA.N,income_statement,"Gross Profit, Industrial Property, Total",0001675149
8,2016-06-30,9.700000e+07,TR.F.SGATot,AA.N,income_statement,"Selling, General and Administrative Expenses, ...",0001675149
9,2016-06-30,9.000000e+07,TR.F.SGAUnclassif,AA.N,income_statement,"Selling, General and Administrative Expenses, ...",0001675149


In [ ]:
# Save to CSV
income_statements.to_csv("../data/income_statements_with_labels.csv", index=False)

---

- Function to fetch most recent company filings for a given CIK and Date

In [3]:
labeled_balance_sheets = pd.read_csv("../data/balance_sheets_with_labels.csv", dtype={"CIK": str})
labeled_income_statements = pd.read_csv("../data/income_statements_with_labels.csv", dtype={"CIK": str})
labeled_cash_flow_statements = pd.read_csv("../data/cash_flow_statements_with_labels.csv", dtype={"CIK": str})

C:\Users\benny\AppData\Local\Temp\ipykernel_21596\2672417944.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  labeled_income_statements = pd.read_csv("../data/income_statements_with_labels.csv", dtype={"CIK": str})
C:\Users\benny\AppData\Local\Temp\ipykernel_21596\2672417944.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  labeled_cash_flow_statements = pd.read_csv("../data/cash_flow_statements_with_labels.csv", dtype={"CIK": str})


In [18]:
bs_uniques = labeled_balance_sheets["CIK"].unique()
is_uniques = labeled_income_statements["CIK"].unique()
cs_uniques = labeled_cash_flow_statements["CIK"].unique()

# Find overlap 
overlap = set(bs_uniques) & set(is_uniques) & set(cs_uniques)
len(overlap)

1490

In [25]:
def get_llm_ratings(cik: str, balance_sheets = None, income_statements = None, cash_flow_statements = None):
    
    """
    Function that returns a DataFrame with LLM ratings for a given CIK.
    For every reporting date, the function fetches the most recent financial statements, i.e.
    - Balance Sheet
    - Cash Flow Statement
    - Income Statement
    and calls the LLM to get a buy/sell/hold recommendation. In order to avoid issues with reports that were filed slightly apart,
    a window of 10 days around a given reporting date is used. This helps to ensure that the LLM has access to all relevant financial information for a given reporting date.
    Furthermore, reports that were filed slightly apart will not lead to recommendations that are based on partial information only and will also not cause multiple
    recommendations that only lie within the window of 10 days around a given reporting date.
    
    Parameters: 
    cik: str, CIK of the company (Can be looked up on the SEC website)
    balance_sheets: Balance Sheets DataFrame with columns: STD Balance Sheet All, FCC Item Name, CIK and Date (among others)
    income_statements: Income Statements DataFrame with columns: STD Income Statement All, FCC Item Name, CIK and Date (among others)
    cash_flow_statements: Cash Flow Statements DataFrame with columns: STD Cash Flow All, FCC Item Name, CIK and Date (among others)
    """
    
    # First filter dfs for input CIK
    balance_sheets = balance_sheets[balance_sheets["CIK"] == cik].copy()
    income_statements = income_statements[income_statements["CIK"] == cik].copy()
    cash_flow_statements = cash_flow_statements[cash_flow_statements["CIK"] == cik].copy()
    
    # Convert the date columns to datetime objects
    for df in [balance_sheets, income_statements, cash_flow_statements]:
        df["Report Date"] = pd.to_datetime(df["Date"])

    # Determine unique dates
    reporting_dates = pd.concat([
        balance_sheets["Report Date"],
        income_statements["Report Date"],
        cash_flow_statements["Report Date"]
    ]).unique()

    # Sort dates just to be safe
    reporting_dates = np.sort(reporting_dates)

    # In order to handle reports, that were filed slighty apart, a window of 10 days around a given reporting date is used
    window = pd.Timedelta(days=10)

    # Loop over reporting dates to obtain LLM ratings
    llm_ratings = []
    for date in reporting_dates:

        # Subset all financial statements for the given dates +- window days
        # Current quarter
        bs = balance_sheets[(balance_sheets["Report Date"] >= date - window) & (balance_sheets["Report Date"] <= date + window)]
        is_ = income_statements[(income_statements["Report Date"] >= date - window) & (income_statements["Report Date"] <= date + window)]
        cf = cash_flow_statements[(cash_flow_statements["Report Date"] >= date - window) & (cash_flow_statements["Report Date"] <= date + window)]

        # If any of the DataFrames is empty, skip this date
        if bs.empty or is_.empty or cf.empty:
            continue

        # Concatenate reports into a string with correct labels
        bs_str = "\n".join(bs.apply(lambda row: f"{row['position_label']}: {row['STD Balance Sheet All']}", axis=1).astype(str))
        is_str = "\n".join(is_.apply(lambda row: f"{row['position_label']}: {row['STD Income Statement All']}", axis=1).astype(str))
        cf_str = "\n".join(cf.apply(lambda row: f"{row['position_label']}: {row['STD Cash Flow All']}", axis=1).astype(str))




        # Call the LLM to get the rating
        response = client.models.generate_content(
            model="gemini-2.5-flash-lite", # "gemini-2.5-flash"
            config=types.GenerateContentConfig(
                temperature=0, # Deterministic ouput
                system_instruction="""You are an experienced, data-driven financial analyst, that provides concise and accurate answers.""",
                
                thinking_config=types.ThinkingConfig(thinking_budget=0),# Disables thinking, but only required for Gemini 2.5
            ),
            
            contents=[f"""
            Based on the following financial reports only, please provide an investment recommendation for the underlying company.
                      
            Balance Sheet: 
            {bs_str}

            Income Statement: 
            {is_str}

            Cash Flow Statement: 
            {cf_str}

            Provide your answer using only one of the following signals: 'strong buy', 'buy', 'hold', 'sell', or 'strong sell'.
            """]
        )

        # Extract rating from the response
        rating = response.text.strip().lower()
        llm_ratings.append({
            "cik": str(cik),  # Ensure CIK is a string
            "date": date,
            "rating": rating
        })
    
    # Convert the list of dictionaries to a DataFrame
    llm_ratings_df = pd.DataFrame(llm_ratings)

    # Convert Report Date to datetime
    llm_ratings_df["date"] = pd.to_datetime(llm_ratings_df["date"]).dt.date

    # Sort by Report Date
    llm_ratings_df.sort_values(by="date", inplace=True)

    # Reset index
    llm_ratings_df.reset_index(drop=True, inplace=True)
    
    return llm_ratings_df

In [26]:
def create_requests(cik: str, balance_sheets = None, income_statements = None, cash_flow_statements = None):
    
    """
    Function that returns a DataFrame with LLM ratings for a given CIK.
    For every reporting date, the function fetches the most recent financial statements, i.e.
    - Balance Sheet
    - Cash Flow Statement
    - Income Statement
    and calls the LLM to get a buy/sell/hold recommendation. In order to avoid issues with reports that were filed slightly apart,
    a window of 10 days around a given reporting date is used. This helps to ensure that the LLM has access to all relevant financial information for a given reporting date.
    Furthermore, reports that were filed slightly apart will not lead to recommendations that are based on partial information only and will also not cause multiple
    recommendations that only lie within the window of 10 days around a given reporting date.
    
    Parameters: 
    cik: str, CIK of the company (Can be looked up on the SEC website)
    balance_sheets: Balance Sheets DataFrame with columns: STD Balance Sheet All, FCC Item Name, CIK and Date (among others)
    income_statements: Income Statements DataFrame with columns: STD Income Statement All, FCC Item Name, CIK and Date (among others)
    cash_flow_statements: Cash Flow Statements DataFrame with columns: STD Cash Flow All, FCC Item Name, CIK and Date (among others)
    """
    
    # First filter dfs for input CIK
    balance_sheets = balance_sheets[balance_sheets["CIK"] == cik].copy()
    income_statements = income_statements[income_statements["CIK"] == cik].copy()
    cash_flow_statements = cash_flow_statements[cash_flow_statements["CIK"] == cik].copy()
    
    # Convert the date columns to datetime objects
    for df in [balance_sheets, income_statements, cash_flow_statements]:
        df["Report Date"] = pd.to_datetime(df["Date"])

    # Determine unique dates
    reporting_dates = pd.concat([
        balance_sheets["Report Date"],
        income_statements["Report Date"],
        cash_flow_statements["Report Date"]
    ]).unique()

    # Sort dates just to be safe
    reporting_dates = np.sort(reporting_dates)

    # In order to handle reports, that were filed slighty apart, a window of 10 days around a given reporting date is used
    window = pd.Timedelta(days=10)

    # Loop over reporting dates to obtain LLM ratings
    requests = []
    for i, date in enumerate(reporting_dates):


        # Determine current date
        q0_date = date
        # Determine date of previous quarter
        qminus1_date = date - pd.DateOffset(months=3)
        # Determine date of q0 -2 
        qminus2_date = date - pd.DateOffset(months=6)
        # Determine date of q0 -3
        qminus3_date = date - pd.DateOffset(months=9)
        print(f"Processing date {i+1}/{len(reporting_dates)}: {date} (Q0: {q0_date}, Q-1: {qminus1_date}, Q-2: {qminus2_date}, Q-3: {qminus3_date})")
        


        # Subset all financial statements for the given dates +- window days
        # Current quarter
        bs = balance_sheets[
            (balance_sheets["Report Date"] >= date - window) &
            (balance_sheets["Report Date"] <= date + window)
        ]
        is_0 = income_statements[
            (income_statements["Report Date"] >= date - window) &
            (income_statements["Report Date"] <= date + window)
        ]
        cf_0 = cash_flow_statements[
            (cash_flow_statements["Report Date"] >= date - window) &
            (cash_flow_statements["Report Date"] <= date + window)
        ]

        # Previous quarter (Q-1)
        is_qminus1 = income_statements[
            (income_statements["Report Date"] >= qminus1_date - window) &
            (income_statements["Report Date"] <= qminus1_date + window)
        ]
        cf_qminus1 = cash_flow_statements[
            (cash_flow_statements["Report Date"] >= qminus1_date - window) &
            (cash_flow_statements["Report Date"] <= qminus1_date + window)
        ]

        # Two quarters ago (Q-2)
        is_qminus2 = income_statements[
            (income_statements["Report Date"] >= qminus2_date - window) &
            (income_statements["Report Date"] <= qminus2_date + window)
        ]
        cf_qminus2 = cash_flow_statements[
            (cash_flow_statements["Report Date"] >= qminus2_date - window) &
            (cash_flow_statements["Report Date"] <= qminus2_date + window)
        ]

        # Three quarters ago (Q-3)
        is_qminus3 = income_statements[
            (income_statements["Report Date"] >= qminus3_date - window) &
            (income_statements["Report Date"] <= qminus3_date + window)
        ]
        cf_qminus3 = cash_flow_statements[
            (cash_flow_statements["Report Date"] >= qminus3_date - window) &
            (cash_flow_statements["Report Date"] <= qminus3_date + window)
        ]

        # If no reports are available for the given date, skip to next date
        if bs.empty or is_0.empty or cf_0.empty:
            continue
        
        # Concatenate reports into strings with correct labels
        bs_str = "\n".join(bs.apply(lambda row: f"{row['position_label']}: {row['STD Balance Sheet All']}", axis=1).astype(str))
        is_str = "\n".join(is_0.apply(lambda row: f"{row['position_label']}: {row['STD Income Statement All']}", axis=1).astype(str))
        cf_str = "\n".join(cf_0.apply(lambda row: f"{row['position_label']}: {row['STD Cash Flow All']}", axis=1).astype(str))

        # Append previous quarters if available — even if just one of IS or CF is present
        if not is_qminus1.empty:
            is_qminus1_str = "\n".join(is_qminus1.apply(lambda row: f"{row['position_label']}: {row['STD Income Statement All']}", axis=1).astype(str))
            is_str += f"\n\nIncome Statement from previous quarter:\n{is_qminus1_str}"
        if not cf_qminus1.empty:
            cf_qminus1_str = "\n".join(cf_qminus1.apply(lambda row: f"{row['position_label']}: {row['STD Cash Flow All']}", axis=1).astype(str))
            cf_str += f"\n\nCash Flow Statement from previous quarter:\n{cf_qminus1_str}"

        if not is_qminus2.empty:
            is_qminus2_str = "\n".join(is_qminus2.apply(lambda row: f"{row['position_label']}: {row['STD Income Statement All']}", axis=1).astype(str))
            is_str += f"\n\nIncome Statement from two quarters ago:\n{is_qminus2_str}"
        if not cf_qminus2.empty:
            cf_qminus2_str = "\n".join(cf_qminus2.apply(lambda row: f"{row['position_label']}: {row['STD Cash Flow All']}", axis=1).astype(str))
            cf_str += f"\n\nCash Flow Statement from two quarters ago:\n{cf_qminus2_str}"

        if not is_qminus3.empty:
            is_qminus3_str = "\n".join(is_qminus3.apply(lambda row: f"{row['position_label']}: {row['STD Income Statement All']}", axis=1).astype(str))
            is_str += f"\n\nIncome Statement from three quarters ago:\n{is_qminus3_str}"
        if not cf_qminus3.empty:
            cf_qminus3_str = "\n".join(cf_qminus3.apply(lambda row: f"{row['position_label']}: {row['STD Cash Flow All']}", axis=1).astype(str))
            cf_str += f"\n\nCash Flow Statement from three quarters ago:\n{cf_qminus3_str}"
            
        contents=[f"""
        Based on the following financial reports only, please provide an investment recommendation for the underlying company.
                    
        Balance Sheet: 
        {bs_str}

        Income Statement: 
        {is_str}     

        Cash Flow Statement: 
        {cf_str}

        Provide your answer using only one of the following signals: 'buy', 'hold' or 'sell'.
        """]
        requests.append({
            "cik": str(cik),  # Ensure CIK is a string
            "date": date,
            "contents": contents
        })


    
    return requests

In [49]:
def get_llm_ratings_with_previous_quarters(cik: str, balance_sheets = None, income_statements = None, cash_flow_statements = None):
    
    """
    Function that returns a DataFrame with LLM ratings for a given CIK.
    For every reporting date, the function fetches the most recent financial statements, i.e.
    - Balance Sheet
    - Cash Flow Statement
    - Income Statement
    and calls the LLM to get a buy/sell/hold recommendation. In order to avoid issues with reports that were filed slightly apart,
    a window of 10 days around a given reporting date is used. This helps to ensure that the LLM has access to all relevant financial information for a given reporting date.
    Furthermore, reports that were filed slightly apart will not lead to recommendations that are based on partial information only and will also not cause multiple
    recommendations that only lie within the window of 10 days around a given reporting date.
    
    Parameters: 
    cik: str, CIK of the company (Can be looked up on the SEC website)
    balance_sheets: Balance Sheets DataFrame with columns: STD Balance Sheet All, FCC Item Name, CIK and Date (among others)
    income_statements: Income Statements DataFrame with columns: STD Income Statement All, FCC Item Name, CIK and Date (among others)
    cash_flow_statements: Cash Flow Statements DataFrame with columns: STD Cash Flow All, FCC Item Name, CIK and Date (among others)
    """
    
    # First filter dfs for input CIK
    balance_sheets = balance_sheets[balance_sheets["CIK"] == cik].copy()
    income_statements = income_statements[income_statements["CIK"] == cik].copy()
    cash_flow_statements = cash_flow_statements[cash_flow_statements["CIK"] == cik].copy()
    
    # Convert the date columns to datetime objects
    for df in [balance_sheets, income_statements, cash_flow_statements]:
        df["Report Date"] = pd.to_datetime(df["Date"])

    # Determine unique dates
    reporting_dates = pd.concat([
        balance_sheets["Report Date"],
        income_statements["Report Date"],
        cash_flow_statements["Report Date"]
    ]).unique()

    # Sort dates just to be safe
    reporting_dates = np.sort(reporting_dates)

    # In order to handle reports, that were filed slighty apart, a window of 10 days around a given reporting date is used
    window = pd.Timedelta(days=10)

    # Loop over reporting dates to obtain LLM ratings
    llm_ratings = []
    for i, date in enumerate(reporting_dates):


        # Determine current date
        q0_date = date
        # Determine date of previous quarter
        qminus1_date = date - pd.DateOffset(months=3)
        # Determine date of q0 -2 
        qminus2_date = date - pd.DateOffset(months=6)
        # Determine date of q0 -3
        qminus3_date = date - pd.DateOffset(months=9)        


        # Subset all financial statements for the given dates +- window days
        # Current quarter
        bs = balance_sheets[
            (balance_sheets["Report Date"] >= date - window) &
            (balance_sheets["Report Date"] <= date + window)
        ]
        is_0 = income_statements[
            (income_statements["Report Date"] >= date - window) &
            (income_statements["Report Date"] <= date + window)
        ]
        cf_0 = cash_flow_statements[
            (cash_flow_statements["Report Date"] >= date - window) &
            (cash_flow_statements["Report Date"] <= date + window)
        ]

        # Previous quarter (Q-1)
        is_qminus1 = income_statements[
            (income_statements["Report Date"] >= qminus1_date - window) &
            (income_statements["Report Date"] <= qminus1_date + window)
        ]
        cf_qminus1 = cash_flow_statements[
            (cash_flow_statements["Report Date"] >= qminus1_date - window) &
            (cash_flow_statements["Report Date"] <= qminus1_date + window)
        ]

        # Two quarters ago (Q-2)
        is_qminus2 = income_statements[
            (income_statements["Report Date"] >= qminus2_date - window) &
            (income_statements["Report Date"] <= qminus2_date + window)
        ]
        cf_qminus2 = cash_flow_statements[
            (cash_flow_statements["Report Date"] >= qminus2_date - window) &
            (cash_flow_statements["Report Date"] <= qminus2_date + window)
        ]

        # Three quarters ago (Q-3)
        is_qminus3 = income_statements[
            (income_statements["Report Date"] >= qminus3_date - window) &
            (income_statements["Report Date"] <= qminus3_date + window)
        ]
        cf_qminus3 = cash_flow_statements[
            (cash_flow_statements["Report Date"] >= qminus3_date - window) &
            (cash_flow_statements["Report Date"] <= qminus3_date + window)
        ]

        # If no reports are available for the given date, skip to next date
        if bs.empty or is_0.empty or cf_0.empty:
            print(f"No reports available for date {date}. Skipping...")
            continue
        
        # Concatenate reports into strings with correct labels
        bs_str = "\n".join(bs.apply(lambda row: f"{row['position_label']}: {row['STD Balance Sheet All']}", axis=1).astype(str))
        is_str = "\n".join(is_0.apply(lambda row: f"{row['position_label']}: {row['STD Income Statement All']}", axis=1).astype(str))
        cf_str = "\n".join(cf_0.apply(lambda row: f"{row['position_label']}: {row['STD Cash Flow All']}", axis=1).astype(str))

        # Append previous quarters if available — even if just one of IS or CF is present
        if not is_qminus1.empty:
            is_qminus1_str = "\n".join(is_qminus1.apply(lambda row: f"{row['position_label']}: {row['STD Income Statement All']}", axis=1).astype(str))
            is_str += f"\n\nIncome Statement from previous quarter:\n{is_qminus1_str}"
        if not cf_qminus1.empty:
            cf_qminus1_str = "\n".join(cf_qminus1.apply(lambda row: f"{row['position_label']}: {row['STD Cash Flow All']}", axis=1).astype(str))
            cf_str += f"\n\nCash Flow Statement from previous quarter:\n{cf_qminus1_str}"

        if not is_qminus2.empty:
            is_qminus2_str = "\n".join(is_qminus2.apply(lambda row: f"{row['position_label']}: {row['STD Income Statement All']}", axis=1).astype(str))
            is_str += f"\n\nIncome Statement from two quarters ago:\n{is_qminus2_str}"
        if not cf_qminus2.empty:
            cf_qminus2_str = "\n".join(cf_qminus2.apply(lambda row: f"{row['position_label']}: {row['STD Cash Flow All']}", axis=1).astype(str))
            cf_str += f"\n\nCash Flow Statement from two quarters ago:\n{cf_qminus2_str}"

        if not is_qminus3.empty:
            is_qminus3_str = "\n".join(is_qminus3.apply(lambda row: f"{row['position_label']}: {row['STD Income Statement All']}", axis=1).astype(str))
            is_str += f"\n\nIncome Statement from three quarters ago:\n{is_qminus3_str}"
        if not cf_qminus3.empty:
            cf_qminus3_str = "\n".join(cf_qminus3.apply(lambda row: f"{row['position_label']}: {row['STD Cash Flow All']}", axis=1).astype(str))
            cf_str += f"\n\nCash Flow Statement from three quarters ago:\n{cf_qminus3_str}"
            

        # Call the LLM to get the rating
        response = client.models.generate_content(
           # model="gemini-2.5-flash", 
            model="gemini-2.5-flash-lite",
            config=types.GenerateContentConfig(
                temperature=0, # Deterministic ouput
                system_instruction="""You are an experienced, data-driven financial analyst, that provides concise and accurate answers.""",
                
                thinking_config=types.ThinkingConfig(thinking_budget=0),# Disables thinking, but only required for Gemini 2.5
            ),
            
            contents=[f"""
            Based on the following financial reports only, please provide an investment recommendation for the underlying company.
                      
            Balance Sheet: 
            {bs_str}

            Income Statement: 
            {is_str}

            Cash Flow Statement: 
            {cf_str}

            Provide your answer using only one of the following signals: 'buy', 'hold' or 'sell'.
            """]
        )

        # Extract rating from the response
        rating = response.text.strip().lower()
        llm_ratings.append({
            "cik": str(cik), 
            "date": date,
            "rating": rating
        })

    # If no ratings were generated, return None
    if not llm_ratings:
        return None
    
    # Convert the list of dictionaries to a DataFrame
    llm_ratings_df = pd.DataFrame(llm_ratings)

    llm_ratings_df["date"] = pd.to_datetime(llm_ratings_df["date"]).dt.date

    # Sort by Report Date
    llm_ratings_df.sort_values(by="date", inplace=True)

    # Reset index
    llm_ratings_df.reset_index(drop=True, inplace=True)
    
    return llm_ratings_df

In [50]:
# Function to get LLM ratings for all CIKs
import pandas as pd
import time
from tqdm import tqdm

def llm_ratings_loop(
    cik_list,
    balance_sheets,
    income_statements,
    cash_flow_statements,
    function_to_use,
    output_path_ratings="../data/ciklist1_ratings_with_previous_quarters.csv",
    output_path_failed="../data/failed_ciks1.csv",
    retries=5,
    retry_delay=30
):
    """
    Process a list of CIKs to retrieve LLM ratings with retry logic on server errors.

    Args:
        cik_list (pd.DataFrame): DataFrame with a "CIK" column.
        balance_sheets (dict or DataFrame): Balance sheet data.
        income_statements (dict or DataFrame): Income statement data.
        cash_flow_statements (dict or DataFrame): Cash flow data.
        function_to_use (callable): Function to call for each CIK. (Either get_llm_ratings_with_previous_quarters or get_llm_ratings)
        output_path_ratings (str): File path to save ratings CSV.
        output_path_failed (str): File path to save failed CIKs.
        retries (int): Number of retry attempts on server error.
        retry_delay (int): Seconds to wait between retries.

    Returns:
        pd.DataFrame: Combined ratings DataFrame.
    """
    list_ratings = []
    failed_ciks = []
    progress_bar = tqdm(cik_list["CIK"], desc="Processing CIKs")

    for i, cik in enumerate(progress_bar):
        progress_bar.set_description(
            f"Processing CIK {i+1}/{len(cik_list)}: {cik} | Time: {pd.Timestamp.now().strftime('%H:%M:%S')}"
        )

        for attempt in range(retries):
            try:
                ratings = function_to_use(
                    cik,
                    balance_sheets,
                    income_statements,
                    cash_flow_statements
                )
                if ratings is not None:
                    list_ratings.append(ratings)
                break  # success, exit retry loop which starts at for attempt in range(retries)

            except ServerError as e:
                print(f"ServerError for CIK {cik} (Attempt {attempt + 1}/{retries}): {e}")
                if attempt < retries - 1:
                    time.sleep(retry_delay)
                else: # This else statement basically only runs if all retries failed i.e. the inner loop is completed, hence it starts with the next CIK
                    failed_ciks.append(cik)

    # Save results to CSV
    cik_ratings_df = pd.concat(list_ratings, ignore_index=True)
    cik_ratings_df.to_csv(output_path_ratings, index=False)

    # Save failed CIKs to CSV
    if failed_ciks:
        pd.Series(failed_ciks).to_csv(output_path_failed, index=False)

    return cik_ratings_df

- Finally, I can use this function to loop over all company CIKs 

In [4]:
# First determine unique CIKs
ciks = set(labeled_balance_sheets["CIK"].unique()) | set(labeled_income_statements["CIK"].unique()) | set(labeled_cash_flow_statements["CIK"].unique())

# Convert to DataFrame to make slicing into equal parts easier
ciks = pd.DataFrame(ciks, columns=["CIK"])

---
Testing the function

This was the first approach of requesting LLM recommendations, before the LLM loop function was implemented. 

In [ ]:
ciks1_ratings = []
progress_bar = tqdm(ciks1["CIK"], desc="Processing CIKs")

for i, cik in enumerate(progress_bar):
    # Progress message
    progress_bar.set_description(f"Processing CIK {i+1}/{len(ciks1)}: {cik} | Time: {pd.Timestamp.now().strftime('%H:%M:%S')}")
    # Get LLM ratings for the given CIK
    ratings = get_llm_ratings(cik, labeled_balance_sheets, labeled_income_statements, labeled_cash_flow_statements)
    ciks1_ratings.append(ratings)
# Concatenate all ratings into a single DataFrame
ciks_test_ratings_df = pd.concat(ciks1_ratings, ignore_index=True)
# Save to CSV
ciks_test_ratings_df.to_csv("../data/ciks1_ratings.csv", index=False)

Processing CIK 100/100: 0001396009 | Time: 18:19:23: 100%|██████████| 100/100 [1:14:11<00:00, 44.52s/it]


In [15]:
ciks2_ratings = []
progress_bar = tqdm(ciks2["CIK"], desc="Processing CIKs")

for i, cik in enumerate(progress_bar):
    # Progress message
    progress_bar.set_description(f"Processing CIK {i+1}/{len(ciks2)}: {cik} | Time: {pd.Timestamp.now().strftime('%H:%M:%S')}")
    # Get LLM ratings for the given CIK
    ratings = get_llm_ratings(cik, labeled_balance_sheets, labeled_income_statements, labeled_cash_flow_statements)
    ciks2_ratings.append(ratings)
# Concatenate all ratings into a single DataFrame
ciks2_ratings_df = pd.concat(ciks2_ratings, ignore_index=True)
# Save to CSV
ciks2_ratings_df.to_csv("../data/ciks2_ratings.csv", index=False)

Processing CIK 100/100: 0001094831 | Time: 16:25:34: 100%|██████████| 100/100 [1:02:45<00:00, 37.65s/it]


In [ ]:
ciks3_ratings = []
progress_bar = tqdm(ciks3["CIK"], desc="Processing CIKs")

for i, cik in enumerate(progress_bar):
    # Progress message
    progress_bar.set_description(f"Processing CIK {i+1}/{len(ciks3)}: {cik} | Time: {pd.Timestamp.now().strftime('%H:%M:%S')}")
    # Get LLM ratings for the given CIK
    ratings = get_llm_ratings(cik, labeled_balance_sheets, labeled_income_statements, labeled_cash_flow_statements)
    ciks3_ratings.append(ratings)
# Concatenate all ratings into a single DataFrame
ciks3_ratings_df = pd.concat(ciks3_ratings, ignore_index=True)
# Save to CSV
ciks3_ratings_df.to_csv("../data/ciks3_ratings.csv", index=False)

In [8]:
ciks4_ratings = []
progress_bar = tqdm(ciks4["CIK"], desc="Processing CIKs")

for i, cik in enumerate(progress_bar):
    # Progress message
    progress_bar.set_description(f"Processing CIK {i+1}/{len(ciks4)}: {cik} | Time: {pd.Timestamp.now().strftime('%H:%M:%S')}")
    # Get LLM ratings for the given CIK
    ratings = get_llm_ratings(cik, labeled_balance_sheets, labeled_income_statements, labeled_cash_flow_statements)
    ciks4_ratings.append(ratings)
# Concatenate all ratings into a single DataFrame
ciks4_ratings_df = pd.concat(ciks4_ratings, ignore_index=True)
# Save to CSV
ciks4_ratings_df.to_csv("../data/ciks4_ratings.csv", index=False)

Processing CIK 100/100: 0000890926 | Time: 17:44:45: 100%|██████████| 100/100 [1:01:25<00:00, 36.85s/it]


In [8]:
ciks5_ratings = []
progress_bar = tqdm(ciks5["CIK"], desc="Processing CIKs")

for i, cik in enumerate(progress_bar):
    # Progress message
    progress_bar.set_description(f"Processing CIK {i+1}/{len(ciks5)}: {cik} | Time: {pd.Timestamp.now().strftime('%H:%M:%S')}")
    # Get LLM ratings for the given CIK
    ratings = get_llm_ratings(cik, labeled_balance_sheets, labeled_income_statements, labeled_cash_flow_statements)
    ciks5_ratings.append(ratings)
# Concatenate all ratings into a single DataFrame
ciks5_ratings_df = pd.concat(ciks5_ratings, ignore_index=True)
# Save to CSV
ciks5_ratings_df.to_csv("../data/ciks5_ratings.csv", index=False)

Processing CIK 100/100: 0001669811 | Time: 16:04:57: 100%|██████████| 100/100 [1:06:02<00:00, 39.63s/it]


In [10]:
ciks6_ratings = []
progress_bar = tqdm(ciks6["CIK"], desc="Processing CIKs")

for i, cik in enumerate(progress_bar):
    # Progress message
    progress_bar.set_description(f"Processing CIK {i+1}/{len(ciks6)}: {cik} | Time: {pd.Timestamp.now().strftime('%H:%M:%S')}")
    # Get LLM ratings for the given CIK
    ratings = get_llm_ratings(cik, labeled_balance_sheets, labeled_income_statements, labeled_cash_flow_statements)
    ciks6_ratings.append(ratings)
# Concatenate all ratings into a single DataFrame
ciks6_ratings_df = pd.concat(ciks6_ratings, ignore_index=True)
# Save to CSV
ciks6_ratings_df.to_csv("../data/ciks6_ratings.csv", index=False)

Processing CIK 100/100: 0001090872 | Time: 20:48:09: 100%|██████████| 100/100 [1:37:41<00:00, 58.61s/it]


---

### Download of LLM ratings including financial data from previous quarters (going back up to 1 year, if possible)

In [ ]:
cik_list1 = ciks[:500]
cik_list2 = ciks[500:1000]
cik_list3 = ciks[1000:]
len(cik_list1) + len(cik_list2) + len(cik_list3) == len(ciks)

1491

In [ ]:
ratings1 = llm_ratings_loop(
    cik_list=cik_list1,
    balance_sheets=labeled_balance_sheets,
    income_statements=labeled_income_statements,
    cash_flow_statements=labeled_cash_flow_statements,
    output_path_ratings="../data/ciklist1_ratings_with_previous_quarters.csv",
    output_path_failed="../data/failed_ciks1.csv",
    retries=5,
    retry_delay=30)

,CIK
0,0000080172
1,0000907242
2,0000745732
3,0000885725
4,0000793733


In [13]:
ratings2 = llm_ratings_loop(
    cik_list=cik_list2,
    balance_sheets=labeled_balance_sheets,
    income_statements=labeled_income_statements,
    cash_flow_statements=labeled_cash_flow_statements,
    output_path_ratings="../data/ciklist2_ratings_with_previous_quarters.csv",
    output_path_failed="../data/failed_ciks2.csv",
    retries=5,
    retry_delay=30)

Processing CIK 8/500: 0000949157 | Time: 11:32:08:   1%|▏         | 7/500 [05:49<7:21:43, 53.76s/it]

ServerError for CIK 0000949157 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 35/500: 0000040533 | Time: 11:56:55:   7%|▋         | 34/500 [30:35<6:31:01, 50.35s/it] 

ServerError for CIK 0000040533 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 65/500: 0000854775 | Time: 12:23:40:  13%|█▎        | 64/500 [57:21<5:16:49, 43.60s/it]

...

Processing CIK 418/500: 0000109198 | Time: 17:43:20:  83%|████████▎ | 417/500 [6:17:01<1:41:09, 73.13s/it]

ServerError for CIK 0000109198 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 495/500: 0000003453 | Time: 18:46:13:  99%|█████████▉| 494/500 [7:19:54<04:26, 44.34s/it]  

ServerError for CIK 0000003453 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 500/500: 0000887343 | Time: 18:51:03: 100%|██████████| 500/500 [7:25:44<00:00, 53.49s/it]


In [10]:
ratings3 = llm_ratings_loop(
    cik_list=cik_list3,
    balance_sheets=labeled_balance_sheets,
    income_statements=labeled_income_statements,
    cash_flow_statements=labeled_cash_flow_statements,
    output_path_ratings="../data/ciklist3_ratings_with_previous_quarters.csv",
    output_path_failed="../data/failed_ciks3.csv",
    retries=5,
    retry_delay=30)

Processing CIK 48/491: 0000058492 | Time: 14:42:36:  10%|▉         | 47/491 [36:03<5:50:44, 47.40s/it]

ServerError for CIK 0000058492 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 98/491: 0001519751 | Time: 15:24:16:  20%|█▉        | 97/491 [1:17:42<6:10:15, 56.38s/it]

ServerError for CIK 0001519751 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 116/491: 0000042582 | Time: 15:42:16:  23%|██▎       | 115/491 [1:35:43<6:20:24, 60.70s/it]

...

Processing CIK 177/491: 0000818479 | Time: 16:59:21:  36%|███▌      | 176/491 [2:52:48<4:58:01, 56.77s/it]

ServerError for CIK 0000818479 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 252/491: 0000005513 | Time: 17:54:04:  51%|█████     | 251/491 [3:47:31<2:05:28, 31.37s/it]

ServerError for CIK 0000005513 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 491/491: 0001792789 | Time: 20:43:24: 100%|██████████| 491/491 [6:37:02<00:00, 48.52s/it]  


---
- Going back to downloading recommendations with only most current financial statements

    - Using get_llm_ratings
    - Using the llm loop function 

In [14]:
ratings1 = llm_ratings_loop(
    cik_list=cik_list1,
    balance_sheets=labeled_balance_sheets,
    income_statements=labeled_income_statements,
    cash_flow_statements=labeled_cash_flow_statements,
    output_path_ratings="../data/ciklist1_ratings_with_most_recent_quarters.csv",
    output_path_failed="../data/failed_ciks1_most_recent_quarters.csv",
    retries=5,
    retry_delay=30)

Processing CIK 5/500: 0001368622 | Time: 13:10:25:   1%|          | 4/500 [03:07<6:46:34, 49.18s/it]

ServerError for CIK 0001368622 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}
ServerError for CIK 0001368622 (Attempt 2/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 10/500: 0000106640 | Time: 13:16:33:   2%|▏         | 9/500 [09:15<8:01:48, 58.88s/it]

ServerError for CIK 0000106640 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 14/500: 0000313927 | Time: 13:21:21:   3%|▎         | 13/500 [14:03<8:17:11, 61.25s/it] 

...

Processing CIK 493/500: 0001551306 | Time: 19:41:44:  98%|█████████▊| 492/500 [6:34:26<09:16, 69.58s/it]

ServerError for CIK 0001551306 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 498/500: 0000936468 | Time: 19:45:43:  99%|█████████▉| 497/500 [6:38:25<02:28, 49.54s/it]

ServerError for CIK 0000936468 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 500/500: 0000042582 | Time: 19:48:06: 100%|██████████| 500/500 [6:41:35<00:00, 48.19s/it]


In [10]:
ratings2 = llm_ratings_loop(
    cik_list=cik_list2,
    balance_sheets=labeled_balance_sheets,
    income_statements=labeled_income_statements,
    cash_flow_statements=labeled_cash_flow_statements,
    output_path_ratings="../data/ciklist2_ratings_with_most_recent_quarters.csv",
    output_path_failed="../data/failed_ciks2_most_recent_quarters.csv",
    retries=5,
    retry_delay=30)

Processing CIK 65/500: 0001235468 | Time: 10:16:44:  13%|█▎        | 64/500 [45:01<4:22:23, 36.11s/it]

ServerError for CIK 0001235468 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 227/500: 0000723188 | Time: 12:03:53:  45%|████▌     | 226/500 [2:32:11<3:42:18, 48.68s/it]

ServerError for CIK 0000723188 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 283/500: 0000046080 | Time: 12:42:48:  56%|█████▋    | 282/500 [3:11:06<2:25:58, 40.18s/it]

...

Processing CIK 470/500: 0000051253 | Time: 14:50:51:  94%|█████████▍| 469/500 [5:19:09<25:10, 48.71s/it]  

ServerError for CIK 0000051253 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 471/500: 0000875320 | Time: 14:52:50:  94%|█████████▍| 470/500 [5:21:08<34:55, 69.85s/it]

ServerError for CIK 0000875320 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 500/500: 0001013857 | Time: 15:17:13: 100%|██████████| 500/500 [5:46:28<00:00, 41.58s/it]


In [11]:
ratings3 = llm_ratings_loop(
    cik_list=cik_list3,
    balance_sheets=labeled_balance_sheets,
    income_statements=labeled_income_statements,
    cash_flow_statements=labeled_cash_flow_statements,
    output_path_ratings="../data/ciklist3_ratings_with_most_recent_quarters.csv",
    output_path_failed="../data/failed_ciks3_most_recent_quarters.csv",
    retries=5,
    retry_delay=30)

Processing CIK 17/491: 0001309108 | Time: 15:31:52:   3%|▎         | 16/491 [13:41<5:42:15, 43.23s/it]

ServerError for CIK 0001309108 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 43/491: 0000801337 | Time: 15:53:52:   9%|▊         | 42/491 [35:41<5:24:34, 43.37s/it]

ServerError for CIK 0000801337 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 48/491: 0000049196 | Time: 15:58:35:  10%|▉         | 47/491 [40:24<5:45:57, 46.75s/it]

...

Processing CIK 467/491: 0000811596 | Time: 21:48:46:  95%|█████████▍| 466/491 [6:30:36<16:27, 39.50s/it]

ServerError for CIK 0000811596 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 478/491: 0000931015 | Time: 21:56:37:  97%|█████████▋| 477/491 [6:38:26<09:59, 42.81s/it]

ServerError for CIK 0000931015 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 491/491: 0000818479 | Time: 22:05:57: 100%|██████████| 491/491 [6:48:37<00:00, 49.93s/it]


---


# Batched API requests

Since the number of individual requests that would be sent using the previously written function would exceed the daily request limit by far, batch mode will be used.

In [9]:
def create_llm_requests(cik_list, labeled_balance_sheets, labeled_income_statements, labeled_cash_flow_statements):
    requests_data = []
    for i, cik in enumerate(cik_list["CIK"]):
        # Progress message
        print(f"Processing CIK {cik} ({i+1}/{len(cik_list)})")
        
        # First filter dfs for input CIK
        balance_sheets = labeled_balance_sheets[labeled_balance_sheets["CIK"] == cik].copy()
        income_statements = labeled_income_statements[labeled_income_statements["CIK"] == cik].copy()
        cash_flow_statements = labeled_cash_flow_statements[labeled_cash_flow_statements["CIK"] == cik].copy()
        
        # Convert the date columns to datetime objects
        for df in [balance_sheets, income_statements, cash_flow_statements]:
            df["Report Date"] = pd.to_datetime(df["Date"])

        # Determine unique dates
        reporting_dates = pd.concat([
            balance_sheets["Report Date"],
            income_statements["Report Date"],
            cash_flow_statements["Report Date"]
        ]).unique()

        # Sort dates just to be safe
        reporting_dates = np.sort(reporting_dates)

        # In order to handle reports, that were filed slighty apart, a window of 10 days around a given reporting date is used
        window = pd.Timedelta(days=10)

        # Loop over reporting dates to obtain LLM ratings
        for date in reporting_dates:

            # Subset all financial statements for the given date +- window days
            bs = balance_sheets[(balance_sheets["Report Date"] >= date - window) & (balance_sheets["Report Date"] <= date + window)]
            is_ = income_statements[(income_statements["Report Date"] >= date - window) & (income_statements["Report Date"] <= date + window)]
            cf = cash_flow_statements[(cash_flow_statements["Report Date"] >= date - window) & (cash_flow_statements["Report Date"] <= date + window)]
        
            # If no reports are available for the given date, skip to next date
            if bs.empty or is_.empty or cf.empty:
                continue
            
            # Concatenate reports into a string with correct labels
            bs_str = "\n".join(bs.apply(lambda row: f"{row['position_label']}: {row['STD Balance Sheet All']}", axis=1).astype(str))
            is_str = "\n".join(is_.apply(lambda row: f"{row['position_label']}: {row['STD Income Statement All']}", axis=1).astype(str))
            cf_str = "\n".join(cf.apply(lambda row: f"{row['position_label']}: {row['STD Cash Flow All']}", axis=1).astype(str))

            # Construct the request for batch processing
            requests_data.append({
                "key": f"request_{i}_{cik}_{pd.Timestamp(date).strftime('%d_%m_%Y')}",
                "request": {
                    "contents": [{
                        "parts": [{
                            "text": f"""
                            Based on the following financial reports only, please provide an investment recommendation for the underlying company.

                            Balance Sheet: 
                            {bs_str}

                            Income Statement: 
                            {is_str}

                            Cash Flow Statement: 
                            {cf_str}

                            Provide your answer using only one of the following signals: 'strong buy', 'buy', 'hold', 'sell', or 'strong sell'.
                            """.strip()  # strip removes leading and trailing whitespace
                        }]
                    }],
                    "generation_config": {"temperature": 0,
                                          "thinking_config":{"thinking_budget": 0}
                    },
                    "system_instruction": {
                        "parts": [{"text": "You are an experienced, data-driven financial analyst, that provides concise and accurate answers."}]
                    }
                }
            })

                     

                    

                      # Disables thinking, but only required for Gemini 2.5
            
    return requests_data


In [10]:
cik_sublist = ciks1[:100]
ciks1_requests = create_llm_requests(cik_sublist, labeled_balance_sheets, labeled_income_statements, labeled_cash_flow_statements)

Processing CIK 0001996862 (1/100)
Processing CIK 0001049502 (2/100)
Processing CIK 0001071739 (3/100)
Processing CIK 0001164863 (4/100)
Processing CIK 0000717605 (5/100)
Processing CIK 0000008947 (6/100)
Processing CIK 0001341439 (7/100)
Processing CIK 0001955520 (8/100)
Processing CIK 0000775158 (9/100)
Processing CIK 0001166003 (10/100)
Processing CIK 0001360901 (11/100)
Processing CIK 0000822416 (12/100)
Processing CIK 0000066740 (13/100)
Processing CIK 0000089089 (14/100)
Processing CIK 0000889331 (15/100)
Processing CIK 0000876427 (16/100)
Processing CIK 0000104894 (17/100)
Processing CIK 0000788784 (18/100)
Processing CIK 0000038777 (19/100)
Processing CIK 0000805676 (20/100)
Processing CIK 0000089439 (21/100)
Processing CIK 0001025378 (22/100)
Processing CIK 0001532961 (23/100)
Processing CIK 0001845815 (24/100)
Processing CIK 0000913142 (25/100)
Processing CIK 0001569650 (26/100)
Processing CIK 0000882184 (27/100)
Processing CIK 0000106640 (28/100)
Processing CIK 0001095565 (29

In [81]:
# Create a sample JSONL file
with open("my-batch-requests.jsonl", "w") as f:
    for req in ciks1_requests:
        f.write(json.dumps(req) + "\n")


In [12]:
# Read in file to check if it was created correctly
with open("../data/my-batch-requests.jsonl", "r") as f:
    requests = [json.loads(line) for line in f]

In [83]:
uploaded_file = client.files.upload(
    file='my-batch-requests.jsonl',
    config=types.UploadFileConfig(display_name='my-batch-requests', mime_type='jsonl')
)

In [84]:
print(f"Uploaded file: {uploaded_file.name}")

Uploaded file: files/14jmhiosat1d


In [14]:
# Assumes `uploaded_file` is the file object from the previous step
file_batch_job = client.batches.create(
    model="gemini-2.5-flash",
    src=uploaded_file.name,
    config={
        'display_name': "file-upload-job-1",
    },
)

print(f"Created batch job: {file_batch_job.name}")


NameError: name 'uploaded_file' is not defined

In [66]:
# Use the name of the job you want to check
# e.g., inline_batch_job.name from the previous step
job_name = file_batch_job.name  # (e.g. 'batches/your-batch-id')
batch_job = client.batches.get(name=job_name)

completed_states = set([
    'JOB_STATE_SUCCEEDED',
    'JOB_STATE_FAILED',
    'JOB_STATE_CANCELLED',
])

print(f"Polling status for job: {job_name}")
batch_job = client.batches.get(name=job_name) # Initial get
while batch_job.state.name not in completed_states:
  print(f"Current state: {batch_job.state.name}")
  time.sleep(15) # Wait for 30 seconds before polling again
  batch_job = client.batches.get(name=job_name)

print(f"Job finished with state: {batch_job.state.name}")
if batch_job.state.name == 'JOB_STATE_FAILED':
    print(f"Error: {batch_job.error}")


Polling status for job: batches/omc1mqgq4vbwmhbe01uzo2oxn09ai4rlc5ok
Job finished with state: JOB_STATE_SUCCEEDED


In [67]:
import json

# Use the name of the job you want to check
# e.g., inline_batch_job.name from the previous step
job_name = file_batch_job.name
batch_job = client.batches.get(name=job_name)

if batch_job.state.name == 'JOB_STATE_SUCCEEDED':

    # If batch job was created with a file
    if batch_job.dest and batch_job.dest.file_name:
        # Results are in a file
        result_file_name = batch_job.dest.file_name
        print(f"Results are in file: {result_file_name}")

        print("Downloading result file content...")
        file_content = client.files.download(file=result_file_name)
        # Process file_content (bytes) as needed
        print(file_content.decode('utf-8'))

    # If batch job was created with inline request
    elif batch_job.dest and batch_job.dest.inlined_responses:
        # Results are inline
        print("Results are inline:")
        for i, inline_response in enumerate(batch_job.dest.inlined_responses):
            print(f"Response {i+1}:")
            if inline_response.response:
                # Accessing response, structure may vary.
                try:
                    print(inline_response.response.text)
                except AttributeError:
                    print(inline_response.response) # Fallback
            elif inline_response.error:
                print(f"Error: {inline_response.error}")
    else:
        print("No results found (neither file nor inline).")
else:
    print(f"Job did not succeed. Final state: {batch_job.state.name}")
    if batch_job.error:
        print(f"Error: {batch_job.error}")


Results are in file: files/batch-omc1mqgq4vbwmhbe01uzo2oxn09ai4rlc5ok
{"response":{"candidates":[{"content":{"parts":[{"text":"Hold"}],"role":"model"},"index":0,"finishReason":"STOP"}],"usageMetadata":{"candidatesTokenCount":1,"totalTokenCount":3960,"promptTokensDetails":[{"modality":"TEXT","tokenCount":3959}],"promptTokenCount":3959},"responseId":"6eN4aIvLOvPB1MkPq4XgyQE","modelVersion":"gemini-2.5-flash"},"key":"request_0_0001295401_30_06_2004"}
{"response":{"candidates":[{"finishReason":"STOP","content":{"parts":[{"text":"Hold"}],"role":"model"},"index":0}],"responseId":"6eN4aOadJO-n1MkPs6OcsAc","usageMetadata":{"promptTokensDetails":[{"modality":"TEXT","tokenCount":4096}],"candidatesTokenCount":1,"totalTokenCount":4097,"promptTokenCount":4096},"modelVersion":"gemini-2.5-flash"},"key":"request_0_0001295401_30_09_2004"}



check differences between temperature and thinking budget, but parameters seem to be working

----


Testing